## units only ftrs : unit counts

##### Taehwan Jeong (xikizima@hotmail.com)

### This contains generator to identify units per each player
### Also it generate unit counts feature for model

### 공통 (Common)

In [1]:
import pickle
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_colwidth = -1

from collections import Counter

In [2]:
# 정규식
import re
cmp = re.compile('(?:Target:\s([A-Za-z]*)\s\[([0-9A-Z]*)\][\s;]*)?Location:\s\(([0-9]*)[.0-9]*,\s([0-9]*)[.0-9]*.*?\)')
units = re.compile('([0-9A-Za-z]*)\s\[([0-9A-Z]*)\]')

### 학습용 (Train)

In [3]:
unit_info = pd.read_csv('./data/unit_info_data.csv')
whole_unit_set = set(unit_info['unit'])
unit_set = unit_info.groupby('species')['unit'].apply(set).to_dict()

In [4]:
df = pd.read_csv('./data/train.csv') #, nrows=1000000)

In [5]:
df = df[df['event'].isin(['Selection'])]
arr_units = df['event_contents'].map(units.findall)
del df['event_contents']
df = pd.concat([df, arr_units], axis=1)

In [6]:
unit_dict = {}
abandoned_units = []
# species_units = {'Z':{}, 'P':{}, 'T':{}}
for idx, data in df.iterrows():
    if idx % 10000
    if not data['game_id'] in unit_dict:
        unit_dict[data['game_id']] = [{}, {}]
    for unit_name, unit_id in data['event_contents']:
        if unit_name and len(unit_name) > 0: # and len(npcs.findall(unit_name)) == 0:
            unit_name = re.sub('Sieged|Burrowed', '', unit_name)
            
            if unit_name in ['SCV', 'Drone', 'Probe']:
                continue
            # check the enemy has it
            if unit_id not in unit_dict[data['game_id']][(data['player'] + 1 ) % 2]:
                if unit_name in unit_set[data['species']]:
                    unit_dict[data['game_id']][data['player']][unit_id] = unit_name
                elif unit_name in whole_unit_set:
                    unit_dict[data['game_id']][(data['player'] + 1) % 2][unit_id] = unit_name
                else:
                    abandoned_units.append(unit_name)
                # save unit names per each species
#                 if unit_name in species_units[data['species']]:
#                     species_units[data['species']][unit_name] += 1
#                 else:
#                     species_units[data['species']][unit_name] = 1
        else:
            print(f"{idx}: [gid {data['game_id']}] Invalid item name {unit_name} ({unit_id})")

In [7]:
# Dict to save units per players in each game
pickle.dump(unit_dict, open('./data/unit_dict.pkl', 'wb'))

In [8]:
# Winner index per game
winner_dict = df.groupby(by=['game_id'])['winner'].first().to_dict()
pickle.dump(winner_dict, open('./data/winner_dict.pkl', 'wb'))

In [9]:
unit_ftr = []
unit_cols = sorted(whole_unit_set)
game_cnt = max(df['game_id']) + 1

for g in range(game_cnt):
    unit_ftr.append([])
    for p in range(2):
        unit_ftr[-1].append({key:0 for key in unit_cols})
        for unit in unit_dict[g][p].values():
            unit_ftr[-1][p][unit] += 1

In [10]:
merged_unit_ftr = []
winner_ftr = []
for GID in range(game_cnt):
    merged_unit_ftr.append(list(unit_ftr[GID][0].values()) + list(unit_ftr[GID][1].values()))
    winner_ftr.append(winner_dict[GID])

In [27]:
ret_raw = pd.DataFrame(merged_unit_ftr, index=range(game_cnt))
ret_raw.columns = unit_cols + unit_cols
ret_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [30]:
ret_raw = ret_raw.reset_index().rename(columns={'index':'game_id'})
ret_raw.to_csv('./data/unit_select_raw_train.csv', index=False)

### 예측용 (Test)

In [31]:
df = pd.read_csv('./data/test.csv') #, nrows=1000000)
df.shape, df['game_id'].min(), df['game_id'].max()

((28714849, 6), 55658)

In [81]:
df.shape, df['game_id'].min(), df['game_id'].max()

((4077893, 6), 38872, 55658)

In [37]:
df = df[df['event'].isin(['Selection'])]
arr_units = df['event_contents'].map(units.findall)
del df['event_contents']
df = pd.concat([df, arr_units], axis=1)

In [52]:
unit_dict = {}
abandoned_units = []
# species_units = {'Z':{}, 'P':{}, 'T':{}}
for idx, data in df.iterrows():
    if not data['game_id'] in unit_dict:
        unit_dict[data['game_id']] = [{}, {}]
    for unit_name, unit_id in data['event_contents']:
        if unit_name and len(unit_name) > 0: # and len(npcs.findall(unit_name)) == 0:
            unit_name = re.sub('Sieged|Burrowed', '', unit_name)
            
            if unit_name in ['SCV', 'Drone', 'Probe']:
                continue
            # check the enemy has it
            if unit_id not in unit_dict[data['game_id']][(data['player'] + 1 ) % 2]:
                if unit_name in unit_set[data['species']]:
                    unit_dict[data['game_id']][data['player']][unit_id] = unit_name
                elif unit_name in whole_unit_set:
                    unit_dict[data['game_id']][(data['player'] + 1) % 2][unit_id] = unit_name
                else:
                    abandoned_units.append(unit_name)
                # save unit names per each species
#                 if unit_name in species_units[data['species']]:
#                     species_units[data['species']][unit_name] += 1
#                 else:
#                     species_units[data['species']][unit_name] = 1
        else:
            print(f"{idx}: [gid {data['game_id']}] Invalid item name {unit_name} ({unit_id})")

In [53]:
pickle.dump(unit_dict, open('./data/unit_dict_test.pkl', 'wb'))

In [54]:
game_id_idx = list(range(df['game_id'].min(), df['game_id'].max() + 1))

In [55]:
unit_ftr = []
unit_cols = sorted(whole_unit_set)
game_cnt = max(df['game_id']) + 1

for g in game_id_idx:
    unit_ftr.append([])
    for p in range(2):
        unit_ftr[-1].append({key:0 for key in unit_cols})
        for unit in unit_dict[g][p].values():
            unit_ftr[-1][p][unit] += 1

In [84]:
merged_unit_ftr = []
for idx in range(len(unit_ftr)):
    merged_unit_ftr.append(list(unit_ftr[idx][0].values()) + list(unit_ftr[idx][1].values()))

In [88]:
ret_raw = pd.DataFrame(merged_unit_ftr, index=game_id_idx)
ret_raw.columns = unit_cols + unit_cols
ret_raw.head()

,Adept,Archon,Baneling,Banshee,Battlecruiser,BroodLord,Carrier,Colossus,Corruptor,Cyclone,...,Tempest,Thor,Ultralisk,Viking,Viper,VoidRay,WarpPrism,WidowMine,Zealot,Zergling
55654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55655,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
ret_raw = ret_raw.reset_index().rename(columns={'index':'game_id'})
ret_raw.to_csv('./data/unit_select_raw_test.csv', index=False)